In [40]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

In [ ]:
#enter the estimates page you would like to scrape into url, requests and bs4 pull the data 

url = "https://www.ontario.ca/page/expenditure-estimates-ministry-health-2024-25"
page = requests.get(url)
soup = BeautifulSoup(page.text,'html')

In [5]:
#extract the tables from the webpage
tables = soup.find_all('table')

In [41]:
#create table1 dataframe
header = tables[0].find_all('th')
rows = tables[0].find_all('tr')
column_headers = [title.text.strip() for title in header]
table1 = pd.DataFrame(columns = column_headers)
caption = tables[0].find('caption')
table_name = caption.text.strip()
clean_text = re.sub(r"\s", "_", table_name )




In [ ]:
#grab row data and put it in that pandas dataframe
for row in rows[1:]:
    row_data = row.find_all('td')
    individual_row_data = [data.text.strip() for data in row_data]
    length = len(table1)
    table1.loc[length] = individual_row_data

#assign
#globals()[clean_text] = table1

In [46]:
Operating_expense

,Vote,Program,Estimates2024-25 $,Estimates2023-24 $,DifferenceBetween2024-25and2023-24 $,Actual2022-23 $
0,1401,Ministry Administration Program,"99,495,400","98,629,400","866,000","84,247,437"
1,1402,Health Policy and Research Program,"1,194,523,000","1,087,790,200","106,732,800","973,343,922"
2,1403,"Digital, Data and Analytics Program","144,049,700","177,985,400","(33,935,700)","142,857,170"
3,1405,Ontario Health Insurance Program,"29,196,948,600","25,061,058,800","4,135,889,800","24,175,671,438"
4,1406,Population and Public Health Program,"1,538,834,000","1,732,857,200","(194,023,200)","1,954,352,293"
5,1412,Provincial Programs and Stewardship,"4,349,676,500","2,877,260,300","1,472,416,200","2,749,482,350"
6,1413,Information systems,"186,968,100","222,374,200","(35,406,100)","246,149,121"
7,1416,Health Services and Programs,"37,789,079,800","34,688,218,900","3,100,860,900","33,991,024,944"
8,,Less: Special Warrants,"4,200,000,000",-,"4,200,000,000",-
9,,Total operating expense to be voted,"70,299,575,100","65,946,174,400","4,353,400,700","64,317,128,675"


In [24]:
table1

,Vote,Program,Estimates2024-25 $,Estimates2023-24 $,DifferenceBetween2024-25and2023-24 $,Actual2022-23 $
0,1401,Ministry Administration Program,"99,495,400","98,629,400","866,000","84,247,437"
1,1402,Health Policy and Research Program,"1,194,523,000","1,087,790,200","106,732,800","973,343,922"
2,1403,"Digital, Data and Analytics Program","144,049,700","177,985,400","(33,935,700)","142,857,170"
3,1405,Ontario Health Insurance Program,"29,196,948,600","25,061,058,800","4,135,889,800","24,175,671,438"
4,1406,Population and Public Health Program,"1,538,834,000","1,732,857,200","(194,023,200)","1,954,352,293"
5,1412,Provincial Programs and Stewardship,"4,349,676,500","2,877,260,300","1,472,416,200","2,749,482,350"
6,1413,Information systems,"186,968,100","222,374,200","(35,406,100)","246,149,121"
7,1416,Health Services and Programs,"37,789,079,800","34,688,218,900","3,100,860,900","33,991,024,944"
8,,Less: Special Warrants,"4,200,000,000",-,"4,200,000,000",-
9,,Total operating expense to be voted,"70,299,575,100","65,946,174,400","4,353,400,700","64,317,128,675"


In [ ]:
able1

,Vote,Program,Estimates2024-25 $,Estimates2023-24 $,DifferenceBetween2024-25and2023-24 $,Actual2022-23 $
